<a href="https://colab.research.google.com/github/Naveenkumar7093/FMML-MODULUS---IIIT-HYDERABAD/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1st question ans

Yes, averaging the validation accuracy across multiple splits can give you more consistent and reliable results when evaluating the performance of a machine learning model. This technique is commonly referred to as "cross-validation." Cross-validation helps address the issue of variability in the performance metrics that can arise from a single random split of your data into training and validation sets.

Here's how cross-validation works and why it provides more consistent results:

Data Splitting: Instead of performing a single random split of your dataset into a training set and a validation set, cross-validation involves dividing the data into multiple subsets or "folds." Typically, a common choice is k-fold cross-validation, where the data is split into k equally sized subsets.

Training and Evaluation: You then train and evaluate your model k times, each time using a different fold as the validation set and the remaining folds as the training set. This means that every data point in your dataset gets a chance to be in the validation set at least once.

Averaging: After running the model k times, you calculate the validation accuracy (or other performance metric) for each run. Finally, you calculate the average of these k performance metrics to obtain a more stable and representative estimate of your model's performance.

By averaging the results over multiple splits, cross-validation helps you mitigate the effects of random variations that might occur when a single random split is used. It provides a more robust assessment of how well your model generalizes to new, unseen data.

Common choices for k include 5-fold and 10-fold cross-validation, but the appropriate value may vary depending on your dataset size and specific problem. In some cases, you might also consider other variations of cross-validation, such as stratified k-fold cross-validation or leave-one-out cross-validation.

In summary, averaging validation accuracy (or other performance metrics) across multiple splits using cross-validation is a good practice to obtain more consistent and reliable estimates of your model's performance, helping you make better-informed decisions about model selection and hyperparameter tuning.




Send a message


Free Research Preview. ChatGPT may produce

2 nd question ans

Cross-validation, while providing a more stable and reliable estimate of a model's performance compared to a single random split, does not directly give you a more accurate estimate of test accuracy. The reason for this is that cross-validation is primarily used to estimate how well your model generalizes to unseen data within your training dataset, not to estimate performance on a completely independent test dataset.

Here's a breakdown of the distinctions:

Cross-validation: Cross-validation estimates how well your model generalizes to different subsets of your training data. It helps you assess the model's consistency and robustness to variations in the data by averaging performance metrics across multiple folds. It provides a good sense of how your model might perform on similar data within the same dataset.

Test Accuracy: Test accuracy, on the other hand, measures how well your model performs on entirely new and unseen data that was not used during training or cross-validation. This is typically a better representation of how your model will perform in real-world applications, where you encounter data from sources you haven't seen before.

In practice, you should use cross-validation to tune hyperparameters, select models, and assess their generalization capabilities within your training dataset. Once you have a final model and set of hyperparameters you are satisfied with, you should evaluate its performance on a separate, independent test dataset to get an accurate estimate of how it will perform in real-world scenarios.

So, while cross-validation is a valuable tool for model selection and hyperparameter tuning, it doesn't replace the need for a dedicated test dataset to provide an accurate estimate of test accuracy. The test dataset serves as a critical step in evaluating the model's real-world performance.




Send a message


Free Research Preview. ChatGPT may produce inacc

3 rd question ans

The number of iterations, also referred to as the number of training epochs, can have a significant effect on the estimate of a model's performance during training and evaluation. In many cases, increasing the number of iterations can lead to a better estimate of the model's performance, but there are caveats to consider:

Underfitting and Overfitting: The number of iterations plays a crucial role in training a model. If you have too few iterations (underfitting), the model may not have sufficient opportunities to learn from the data, and its performance on both the training and validation sets may be subpar. Conversely, if you have too many iterations (overfitting), the model may memorize the training data and perform poorly on new, unseen data.

Learning Curves: Typically, as you increase the number of iterations, you may observe changes in the learning curves. In the beginning, the training and validation performance may improve as the model learns from the data. However, after a certain point, the validation performance might start to degrade due to overfitting, while the training performance continues to improve. The point where the validation performance begins to degrade is a good indicator of when to stop increasing the iterations.

Computational Resources: Training a model with a high number of iterations can be computationally expensive, especially for deep learning models with many parameters. You should balance the computational resources available with the benefits gained from additional iterations.

Early Stopping: To mitigate overfitting and make better use of computational resources, you can implement techniques like early stopping. Early stopping involves monitoring the validation performance during training and stopping training when the validation performance starts to degrade. This prevents the model from overfitting.

In summary, the number of iterations can impact the estimate of a model's performance. However, the relationship is not linear, and there's often an optimal number of iterations that balances underfitting and overfitting. It's essential to monitor learning curves, use techniques like early stopping, and perform hyperparameter tuning to find the right number of iterations for your specific problem and dataset. Simply increasing the number of iterations indefinitely does not necessarily lead to a better estimate, as it can lead to overfitting and increased computational costs.




Send a message


Free Research Preview. ChatGPT may produce inaccurate informa

4 th question ans

Increasing the number of iterations can help to some extent when dealing with very small training or validation datasets, but it is not a guaranteed solution to overcome the limitations associated with small datasets. When working with limited data, there are several considerations to keep in mind:

Overfitting: If you have a very small training dataset and you increase the number of iterations significantly, your model might start to overfit the training data. Overfitting occurs when the model becomes too complex and starts to memorize the training examples rather than learning to generalize from them. This can lead to poor performance on unseen data, including the validation dataset.

Data Augmentation: Instead of solely relying on more iterations, you can augment your small dataset. Data augmentation techniques involve creating new training examples by applying transformations (e.g., rotations, translations, or flips) to your existing data. This can effectively increase the effective size of your training dataset and help the model learn more robust features.

Regularization: Use regularization techn